# Системи Data Science. Лабораторна робота №1. 
## Назва: Прогнозування рейтингу проектів японської мультиплікації
## Виконала: Сохацька Софія Максимівна, група КМ-81.
## 1.1. Розуміння бізнесу - розуміння даних.

### 1.1.1. Визначення   предметної   області   для   дослідження   і   попередня постановка  задачі  моделюванняв  рамках  конкретного  контексту  з обраної предметної області
В якості предеметної області для дослідження у даній лабораторній роботі була вибрана індустрія японської мультиплікації.

Індустрія японської мультиплікації, в силу культурних особливостей, у яких вона виникла і розвивалася, має досить значні відмінності від більш звичної нам індустрії, вона має свої унікальні жанри, формати і напрями. Метою даного дослідження є вивчення особливостей японської мультиплікації, найпопулярніших жанрів, категорій чи сезонів для виходу, встановлення залежностей між різноманітними характеристиками та їх вплив на популярність та рейтинг твору. Основною задачею є побудова системи прогнозування рейтингу проекту на основі наявних даних, використовуючи досліджені залежності.

Дане дослідження може мати широке застосування у індустрії створення японської мультиплікації для більшого розуміння потреб ринку для того щоб створювати більш надійний та цікавий нинішній публіці продукт.

### 1.1.2. Збір/вибір сирих даних з будь-яких відкритих ресурсів
База данних, що використовується для даного дослідження, містить в собі інформацію про певні роботи в жанрі японської мультиплікації та інформацію про глядачів ціх творів, взятих з платформи під назвою **_"MyAnimeList"_**. Цей датасет був взятий з ресурсу **_Kaggle_**. Робоча директорія: 

In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Для роботи нам потрібен CSV-файл, адреса якого вказана четвертим у списку.

### 1.1.3 Збереження ресурсів даних які потрібні для вирішення конкретної задачі в певному форматі 
Виконаємо завантаження необхідних нам даних в датафрейм, з яким ми будемо працювати, скориставшись пакетом для роботи з CSV-файлами. Продемонстуємо результат, вивевши перші п'ять записів.

In [ ]:
df = pd.read_csv("/kaggle/input/anime-recommendation-database-2020/anime.csv")
df.head(5)

### 1.1.4 Візуалізація
Перейдемо до візуалізації даних. Скориставшись вбудованою функцією Pandas ``````info()``````, виведемо загальну інформацію про наш датафрейм, а саме про загальну кількість записів, наявні поля та тип даних, що в них зберігається.

In [ ]:
df.info()

## 1.2. Підготовка даних
### 1.2.1. Первинна очистка даних
Перед тим, як перейти до приведення даних, для початку позбавимося від даних, що не предствляють для нас цінності. Почнемо з видалення дублікатів.

In [ ]:
df = df.drop_duplicates()
df.info()

Як ми можемо бачити, кількість записів в таблиці не змінилася, це означає що дублікатів у нашій таблиці виявлено не було. Також слід зазначити, що в кожному стовпці к-ть non-null значень співпадає з кількістю записів, тож робимо висновок, що Null і NaN значень немає. 

Для впевненості перевіримо це твердження:

In [ ]:
df.isnull().sum().sum()

Не зважаючи на те, що в нашій таблиці немає ``````Null`````` та ``````NaN`````` значень, в деяких стовпцях присутні такі значення, як ``````Uknown``````. Для початку подивимося, скільки записів містять це значення для кожного поля.

In [ ]:
def Unknown_num(df, name):
    N_Uknown = df[df[name] == "Unknown"]
    print(f"Quantity of Unknown values in {name}:", len(N_Uknown))

for i in df.columns.values:
    Unknown_num(df, i)

В першу чергу слід виділити такі поля, як `````Licensors`````, `````Producers````` та `````Premiered`````, що містять занадто багато значень ``````Uknown`````` порівняно із загальною кількістю записів. Це призводить до знецінення будь-якої інформації в цих полях як для використання в досліженні, так і в подальшому прогнозуванні даних. Тож ці поля слід видалити в розділі конструювання ознак.

Також досить багато значень ``````Uknown`````` у полі ``````English name``````, проте пізніше ми використаємо цей факт на свою користь, тож поки що залишимо все як є.

В першу чергу наявність ``````Uknown`````` важливо прибрати в полях, що мають містити числовий формат даних, адже це буде завадити корректному переведенню даних. Переведення в числовий тип даних потребують такі стовпці, як:
* ``````Score``````
* ``````Episodes``````
* ``````Ranked``````
* ``````Score-10``````
* ``````Score-9``````
* ``````Score-8``````
* ``````Score-7``````
* ``````Score-6``````
* ``````Score-5``````
* ``````Score-4``````
* ``````Score-3``````
* ``````Score-2``````
* ``````Score-1``````

Проте для нас в даному випадку більш важливою є наявність даних саме останніх десяти полів, адже за їх наявності ми зможемо підрахувати значення ``````Score``````. Також, поле ``````Ranked`````` по суті містить в собі значення місця, яке займає певний запис в загальному рейтингу, що базується на значенні ``````Score``````. Цю інформацію можна відтворити по значенням поля ``````Score``````, використовуючи вбудовані функції Pandas. Тож якщо після здійнення очиски даних по всіх інших полях в полі ``````Ranked`````` все ще будуть присутні значення ``````Uknown``````, це поле може бути видалене, і за необхідності відтворене заново.

Тим не менш, поле  ``````Episodes`````` ніяк не може бути заповнене за значеннями інших полів.

Так як пізніше ми будемо прогнозувати параметр ``````Score``````, заміна в вищеописаних полях невистачаючих значень на середні призведе до погіршення якості статистичних даних, що будуть отримані. Також, кількість записів дозволяє нам знехтувати записами, що містять значення Uknown. Тож видалимо ці записи для вказаних полів та ще раз перевіримо наявність записів зі значенням ``````Uknown``````.

Для більшої читабельності, не будемо виводити інформацію про кількість записів зі значенням ``````Uknown`````` для тих полів, для яких вже встановлена їх нульова кількість.

In [ ]:
def Drop_Unknown (df, name):
    index_Unknown= df[df[name] == "Unknown"].index
    df.drop(index_Unknown , inplace=True)
    
to_drop_list = ["Score-10", "Score-9", "Score-8", "Score-7", "Score-6", "Score-5", "Score-4", "Score-3", "Score-2", "Score-1", "Episodes"]

for i in to_drop_list:
    Drop_Unknown(df, i)
    
to_show_list = ["Score", "Genders", "Japanese name", "Type", "Episodes", "Aired", "Studios", "Source", "Duration", "Rating", "Ranked", "Score-10", "Score-9", "Score-8", "Score-7", "Score-6", "Score-5", "Score-4", "Score-3", "Score-2", "Score-1"]
    
for i in to_show_list:
    Unknown_num(df, i)

Помітимо, що в процесі видалення записів по зазначених полях, в нас заодно очистилися поле ``````Type``````. Проте, інформації для полів ``````Score`````` та ``````Ranked`````` все ж не вистачає. 

Замінимо відсутні значення поля ``````Score``````, використовуючи інформацію з інших полів. 

Поле ``````Ranked`````` видалимо пізніше в розділі конструювання ознак.

Для підрахунку, заздалегіть переведемо поля ``````Score-1-10`````` до числового формату.

In [ ]:
df["Score-10"] = df["Score-10"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-9"] = df["Score-9"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-8"] = df["Score-8"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-7"] = df["Score-7"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-6"] = df["Score-6"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-5"] = df["Score-5"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-4"] = df["Score-4"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-3"] = df["Score-3"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-2"] = df["Score-2"].apply(lambda x: x.replace(".0", "")).astype('int')
df["Score-1"] = df["Score-1"].apply(lambda x: x.replace(".0", "")).astype('int')
df.info()

Підрахунок нових значень поля:

In [ ]:
counted_score = df.apply(lambda x: round((x["Score-10"]*10 + x["Score-9"]*9 + x["Score-8"]*8 + x["Score-7"]*7 + x["Score-6"]*6 + x["Score-5"]*5 + x["Score-4"]*4 + x["Score-3"]*3 + x["Score-2"]*2 + x["Score-1"])/(x["Score-10"]+x["Score-9"]+x["Score-8"]+x["Score-7"]+x["Score-6"]+x["Score-5"]+x["Score-4"]+x["Score-3"]+x["Score-2"]+x["Score-1"]), 2), axis=1)

indexScore = df[df["Score"] == "Unknown"].index
for i in indexScore:
    df.loc[i,"Score"] = counted_score[i]

Перевіримо, чи всі записи зі значенням ``````Uknown`````` оновилися.

In [ ]:
Unknown_num(df, "Score")

Також заздалегідь очистимо три поля, в яких значення `````Unknown````` допустимі, проте які ми точно будемо використовувати та в яких кількість "невідомих" записів досить мала, щоб ми могли без докорів сумління видалити їх для більшої чистоти даних.

In [ ]:
to_drop_list = ["Genders", "Aired", "Rating"]

for i in to_drop_list:
    Drop_Unknown(df, i)
    
to_show_list = ["Genders", "Japanese name", "Aired", "Producers", "Studios", "Source", "Duration", "Rating"]
    
for i in to_show_list:
    Unknown_num(df, i)

Для всіх інших полів наявність значень ``````Uknown`````` поки що не є критичною. До того ж ми поки що не знаємо, які поля нам знадобляться, а які ні. Тож щоб уникнути зайвого видалення даних, поки що залишимо все як є, і після проведення розвідувального аналізу за необхідністю виконаємо ще одну чистку.

In [ ]:
df.info()

Після того, як ми впевнилися, що в наших даних немає дублікатів, ``````Null`````` та``````NaN`````` елементів та перевірили запіси, що містять значення ``````Unknown``````, продовжимо приведення даних до стандартних форматів. Не будемо виконувати приведення до стандартних форматів полів, що ми вже зазначили як ті, що потребують видалення.

In [ ]:
df["Score"] = df["Score"].astype('float64')
df["Episodes"] = df["Episodes"].astype('int')

df.info()

In [ ]:
df

Слід зазначити, що незважаючи на те, що більш логічним було б приведення даних в полях `````Genders````` та `````Studios````` до типу `````list`````, проте для розвідувального аналізу даних нам буде зручніше працювати з ними у форматі рядку. Тож відкладемо приведення даних в цих полях до розділу конструювання ознак.

### 1.2.2. Розвідувальний аналіз даних
Скористаємося вбудованою функцією Pandas `````describe()````` для виведення основних статистичних характеристик.

In [ ]:
df.describe()

Заздалегідь підготуємо функції обробки даних для виведення.

Перша функція потрібна нам для для групування категорій, кількість записів в якіх замала для виведення на графіку, в одну окрему групу. Ця функція потрібна для підвищення зрозумілості створюваних діаграм.

Друга функція служить для сортування даних для більш зручного виведення графіків.

Обидві функції працюють з даними, представленими у вигляді словників.

In [ ]:
# Функція для групування замалих категорій, що працює з даними, представленими у вигляді словників
def group_small_data(dct, percent):
    v_sum = 0
    for v in dct.values():
        v_sum+=v
        
    other = 0
    to_del = []
    for k, v in dct.items():
        if v < v_sum*percent:
            other += v
            to_del.append(k)
    for k in to_del:
        del dct[k]
    dct["Other"]=other
    return dct
    
    
# Функція сортування словнику за значеннями в порядку спадання
def sorted_dict(dct):
    sorted_dct = {}
    sorted_keys = sorted(dct, key=dct.get, reverse=True) 

    for i in sorted_keys:
        sorted_dct[i] = dct[i]    
    return sorted_dct

Так як ми збираємося прогнозувати параметр `````Score`````, подивимося на розподіл цієї характеристики, щоб перевірити, чи немає в ньому якихось аномалій.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 8))
plt.hist(df['Score'], bins = 100, edgecolor = 'k');
plt.xlabel('Score'); 
plt.title('Score Distribution');

Маємо нормальний розподіл зі зміщеннням вправо. Ніяких значний аномалій виявнено не було.

Перейдемо до пошуку взаємозв'язків.

Для початку виведемо кругову діаграму жанрів, щоб виявити найпопулярніші з них.

In [ ]:
plt.style.use('seaborn-whitegrid')
from collections import Counter
genres = Counter(','.join(df["Genders"]).replace(' ','').split(','))
g = group_small_data(sorted_dict(genres), 0.01)

plt.figure(figsize=(12, 12))
vals = g.values()
labels = g.keys()
plt.title("Кругова діаграма жанрів")
plt.pie(vals, labels=labels, autopct='%1.1f%%')
plt.show()

Як ми бачимо, очікувано найпопулярнішими жанрами є більш загальні жанри типу `````Комедія````` та `````Пригоди`````, які зазвичай супроводжуються ще одним, більш нішевим жанром.

Тепер побудуємо кругову діаграму, що репрезентує формати, у виді яких подається анімація.

In [ ]:
plt.figure(figsize=(12, 12))
t = df["Type"].value_counts().to_dict()
vals = t.values()
labels = t.keys()
plt.title("Кругова діаграма типів контенту")
plt.pie(vals, labels=labels, autopct='%1.1f%%')
plt.show()

Як ми бачимо, найпопулярнішим жанром є ТV-шоу, що є досить очікуваним. Однак цікавим є те, що другим по популярності форматом є саме OVA, а не фільми. Слід зазначити, що формат OVA представляє собою багатосерійний фільм, зазвичай близько 6-ти серій, що випускається одразу на фізичних носіях та не призначений для транслювання по телебаченню. Такий високий процент може бути досягнутий за рахунок великої популярності такого формату у минулому столітті, коли був популярний перегляд фільмів та серіалів на VHS кассетах.

Перевіримо залежність між форматом виходу та розподілом оцінок користувачів.

In [ ]:
import seaborn as sns
type_list = list(df["Type"].value_counts().index)
plt.figure(figsize=(13, 6.5))
for type_n in type_list:
    subset = df[df["Type"] == type_n]
    sns.kdeplot(subset["Score"], label = type_n, alpha = 0.8);
plt.legend()
plt.xlabel('Score'); plt.ylabel('Розподіл');

In [ ]:
sns.catplot(x="Score", y="Type",  aspect = 3.4,  data=df)

Як бачимо, розподіли для різних форматів значно відрізняються один від одного, то ж ця характеристика має значний вплив на оцінку проекту. Найвищі показники у найпопулярніших форматів: фільмів і телевізійних серіалів. Ми можемо побачити певну залежність між розповсюдженістю форматів та розподілом оцінок - чим більш розповсюджений формат, тим вище в загальному оцінки шоу таких форматів.

Виведемо діаграму, що демонструє, яку частку в індустрії займають різні анимаційні студї. Діаграма виводиться без урахування записів `````Unknown`````

In [ ]:
studios = Counter(','.join(df["Studios"]).replace(' ','').split(','))
st = group_small_data(sorted_dict(studios), 0.011)
del st["Unknown"]
plt.figure(figsize=(12, 12))
vals = st.values()
labels = st.keys()
plt.title("Кругова діаграма студій, що створюють японську мультиплікацію")
plt.pie(vals, labels=labels, autopct='%1.1f%%')
plt.show()

Як бачимо,більшість творів випущені невеликими студіями, що створили порівняно небагато проектів за період свого існування. Тим не менш, більше четверті всіх проектів випущені десяткою найбільших студій.

Наступна гістограма демонструє, розподіл студій за кількістю випущених творів.

In [ ]:
studios_count = sorted_dict(studios)
del studios_count["Unknown"]
st_val_arr = studios_count.values()
plt.figure(figsize=(20, 10))
n_bins = np.arange(1, 700, 5)
plt.hist(st_val_arr, bins=n_bins, edgecolor = "black")
plt.show()

Ця гістограмма підтверджує, що переважна більшість студій випустила за період свого існування менше п'яти проектів, і лише порівняно мала кількість студій подолали цей поріг.

Побудуємо діаграму, що показує розподіл першоджерел японської анімації по категоріях. Діаграма виводиться без урахування записів `````Unknown`````

In [ ]:
plt.figure(figsize=(13, 13))
s = group_small_data(df["Source"].value_counts().to_dict(), 0.016)
del s["Unknown"]
vals = s.values()
labels = s.keys()
plt.title("Кругова діаграма джерел")
plt.pie(vals, labels=labels, autopct='%1.1f%%')
plt.show()

Як ми бачимо, лише третина випущених проектів становить оригінальна анімація. Переважна більшість творів зроблені на основі інших першоджерел, найпопулярнішим із яких є манга. Манга, в свою чергу, є як найпопулярнішим з першоджерел, так і найпопулярнішим джерелом в принципі.

Дослідимо залежність середньої оцінки проектів в залежності від їх першоджерела

In [ ]:
source_list = list(df["Source"].value_counts().index)
plt.figure(figsize=(13, 6.5))
for source_n in source_list:
    subset = df[df["Source"] == source_n]
    sns.kdeplot(subset["Score"], label = source_n, alpha = 0.8);
plt.legend()
plt.xlabel('Score'); plt.ylabel('Розподіл'); 

In [ ]:
sns.catplot(x="Score", y="Source",  aspect = 3.4,  data=df)

За графіками розподілу бачимо, що ця характеристика також має значний вплив на оцінку проекту. В загальному, найвищі оцінки ставлять саме проектам, зробленим на основі манги в якості першоджерела. Також, більш високі оцінки загалом ставлять проектам на основі новелл та візуальних новелл. Найризикованішим вибором є створення оригінального проекту, ми можемо побачити це по найбільшому розкиданню можливих оцінок. Ця статистика пояснює розподіл жанрів, який ми спостерігали на круговій діаграммі: студії більш впевнені в створенні проектів, заснованих на успішних першоджерелах, і більш вірогідно вибирають шлях створення анімації на основі вже існуючої манги, новелли або інших першоджерел. Проте оригінальні проекти також мають великий шанс зацікавити глядача новими історіями, тож цей шлях також є досить популярним, незважаючи на свої ризики.

Тепер подивимося на розподіл рейтингів проектів за віком.

In [ ]:
plt.figure(figsize=(13, 13))
bar_data = df["Rating"].value_counts()
plt.title("Кругова діаграма рейтингів")
plt.pie(bar_data.values, labels=bar_data.index, autopct='%1.1f%%')
plt.show()

Як бачимо, найрозповсюдженішим рейтинг є `````PG-13`````. Це і не дивно, адже цей рейтинг охоплює найбільшу кількість глядачів (хоч рейтинг `````G - All Ages````` і охоплює всі вікові категорії, проте зазвичай такий рейтинг мають дитячі фільми та передачі, які більш дорослі глядачі не захочуть дивитися). 

Виведемо графік залежності аудиторії проекту від його рейтингу. 

In [ ]:
sns.catplot(x="Rating", y="Members",  aspect = 3.4,  data=df)
plt.show()

Очікувано високі показники демонструє категорія рейтингу `````PG-13`````. Щодо рейтингу `````G`````, можемо бачити підтвердження вищевказаної тези про цю категорію - її аудиторія досить мала в порівнянні з тим, яка могла б бути в перспетиві за відсутнісю вікових обмежень. Проте різниця між кільскістю аудиторії по цьому рейтингу та загальної кількості проектів, що отримали такий рейтинг, занадто разюча. Це може пояснюватися тим, що база даних була взята з інтернет-платформи _"MyAnimeList"_, яким не користується основна аудиторія шоу з рейтингом `````G`````.

Неочікувано високу популярність мають більш дорослі рейтинги `````R-17+````` та `````R+`````. Такі показники,у купі з показниками інших категорій рейтингів, можуть свідчити про загальну "дорослість" аудиторії не так японської анімації, як скоріше, платформи _"MyAnimeList"_ в цілому

Тепер перевіримо залежність оцінки проектів від рейтингу.

In [ ]:
rating_list = list(df["Rating"].value_counts().index)
plt.figure(figsize=(13, 6.5))
for rating_n in rating_list:
    subset = df[df["Rating"] == rating_n]
    sns.kdeplot(subset["Score"], label = rating_n, alpha = 0.8);
plt.legend()
plt.xlabel('Score'); plt.ylabel('Розподіл'); 

In [ ]:
sns.catplot(x="Rating", y="Score",  aspect = 3.4,  data=df)
plt.show()

І знову бачимо досить високу залежність оцінок від даного параметру. Найгірше себе проявив рейтинг `````G`````, проте враховуючи попередні результати, такий результат не є дивуючим. Все ще високі показники показують категорії `````PG-13````` та `````R-17+`````.

Подивимося тепер на десятку найвисокооцінених проектів.

In [ ]:
df.nlargest(10, "Score")[["Name", "Score", "Genders", "Type", "Episodes", "Aired", "Studios", "Source", "Rating", "Members"]]


На цьому прикладі бачимо підтвердження висновків, отриманих з досліджень окремих характеристик проектів. В цьому топі переважають проекти формату `````TV````` з рейтингом `````PG-13````` чи `````R-17+`````, та переважна більшість з них має першоджерело-мангу. Ці проекти різних жанрів та від різних студій. Також тут не помітно ніяких залежностей від кількості епізодів шоу. Проте можна побачити, що переважна кількість проектів відносно нові і буди зроблені після 2010 року. Це наштовхує нас додатково перевірити цю характеристику.

### 1.2.3. Попереднє конструювання ознак для продовження розвідувального аналізу даних
По-перше, в нашому датафреймі є одразу три поля, що містять в собі назву об'єкту. Це поля `````Name`````, `````English name````` та `````Japanese name`````. Нам не потрібні одразу три поля для назви, адже на основі цих даних ми не зможемо нічого дізнатися, а для ідентифікації об'єкту нам буде достатньо лише одного поля. Проте наявність чи відсутність англійської назви може бути пов'язана з поширеністю перного твору японської мультиплікації не лише у межах Японії, але й за кордоном, що є цікавим для нашого дослідження. 

Також в нас є поле `````Aired`````, що містить в собі дати початку та закінчення виходу серіалу, чи просто дату виходу, у випадку фільму. Проте інформація в ньому подана у дуже незручному для аналізу вигляді.

Для кращого розуміння нижче наведений приклад записів по вищевказаних полях.

In [ ]:
df[["Name", "English name", "Japanese name", "Aired"]].head(10)

Тож для більш зручної роботи сконструююємо наступні нові характеристики:
* `````Eng_name_availability````` на основі поля `````English name`````, що буде містити одиницю за наявності англійської назви та 0 за її відсутності
* `````Year````` на основі поля`````Aired`````, що буде містити інформацію про рік виходу твору
* `````Season````` на основі поля `````Aired`````, що буде містити сезон,під час якого вийшов твір (зима, весна, літо чи осінь).

Останнє групування виходу по сезонах року зумовлене особливістю індустрії японської анімації, в якій виникла певна традиція "сезонності" проектів, при якій багатосерійні проекти починають і закінчують виходити майже в один і той же час в рамках певного сезону, і багато обговорень щодо японської мультиплікації будується саме на обговоренні проектів певного сезону.

Конструювання поля `````Eng_name_availability`````:

In [ ]:
df["Eng_name_availability"] = 1 

indexEng_name = df[df["English name"] == "Unknown"].index

for i in indexEng_name:
    df.loc[i,"Eng_name_availability"] = 0

Конструювання полів `````Year````` та `````Season````` за допомогою регулярних виразів:

In [ ]:
month_pattern = "([A-Za-z]{3})"
year_pattern = "(\d{4})"
df1 = df.Aired.str.extract(month_pattern, expand=True)
df2 = df.Aired.str.extract(year_pattern, expand=True)
df["Year"] = df2
df["Year"] = df["Year"].astype('int64')
df["Season"] = df1
df["Season"] = df["Season"].map({"Mar": "Spring","Apr": "Spring","May": "Spring","Jun": "Summer","Jul": "Summer","Aug": "Summer", "Sep": "Fall", "Oct": "Fall", "Nov": "Fall", "Dec": "Winter", "Jan": "Winter", "Feb": "Winter" })
df.info()

Як ми бачимо, в полі `````Season````` є  `````NaN````` або`````Null`````. Перевіримо ці записи, щоб дізнатися причину.

In [ ]:
df[df["Season"].isnull()][["Name", "Type", "Aired","Season", "Year"]]

Отже, наш регулярний вираз для вичленення місяця не спрацював, адже в даних записах просто відсутня ця інформація, наявний тільки рік виходу. Видалимо ці записи.

In [ ]:
indexSeasonNaN = df[df["Season"].isnull()].index
df.drop(indexSeasonNaN , inplace=True)

df["Season"].isnull().sum()

### 1.2.4. Продовження розвідувального аналізу даних

Тепер продовжимо розвідувальний аналіз даних, використовуючи нові рахактеристики. Для початку виведемо графік, що демонструє залежність розміру аудиторії проекту від наявності англійської назви.

In [ ]:
sns.catplot(x="Eng_name_availability", y="Members",  aspect = 3.0,  data=df)
plt.show()

Наявність англійської назви дуже сильно корелює з популярнісю проекту. Цьому є дуже просте пояснення - для проектів, які планують випускати на аудиторію по всьому світу, дуже необхідна локалізована назва, що б аудиторії інших країн було зручніше обговорювати проект, зручніше шукати інформацію про нього в інтернеті та, найголовніше, поширювати інформацію про нього. Адже не знаючи мову дуже складно сприйняти та запам'ятати іноземну назву. І навіть якщо хтось захоче розповісти про проект, що його зацікавив, іншим, йому буде зручніше описати його як "той самий проект, де...", а не згадувати точну назву. А для маркетологів проекту це означає що людям буде важче зрозуміти про що йде мова, важче знайти цей проект, а як наслідок продукт досягне меншої кількості людей.

Тепер дослідимо, чи є залежність між сезоном виходу проектів та кількістю проектів, які вийшли в цьому сезоні.

In [ ]:
plt.figure(figsize=(13, 13))
bar_data = df["Season"].value_counts()
plt.title("Кругова діаграма сезонів виходу")
plt.pie(bar_data.values, labels=bar_data.index, autopct='%1.1f%%')
plt.show()

Як бачимо, розподіл майже однаковий по всіх сезонах. Навіть порівнюючи найбільшу категорію `````Spring````` та найменшу `````Winter`````, різниця між ними не настільки значна, що б брати її до уваги. 

Проте ще може бути залежність між сезоном виходу та розподілом аудиторії проектів. Перевіримо цю гіпотезу.

In [ ]:
sns.catplot(x="Season", y="Members",  aspect = 3.0,  data=df)
plt.show()

In [ ]:
sns.catplot(x="Type", y="Year",aspect = 2.0, height=10, data=df)
plt.show()

На цьому графіку наглядно можна побачити історію розвитоку форматів японської анімації з часом. І дійсно, в період з 1980 по 2000 рік найбільш поширеним форматом був саме формат `````OVA`````. Проте пізніше почати набирати популярності і інші формати.

Наостанок, виведемо `````Corellation heatmap````` та проаналізуємо залежності між характеристиками.

In [ ]:
plt.figure(figsize=(20, 8))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True, cmap=sns.diverging_palette(240, 16, n=9, as_cmap=True))
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)
plt.show()

Загальна картина дуже перспективна для майбутнього прогнозування даних. Більше половини параметрів приймає значення від 0.5 до 1. Найгірші показники у поля `````Episodes````` та`````MAL_ID`````. У випадку останнього це не дивно - це унікальний ідентифікатор кожного запису. А ось низькі показники поля `````Episodes````` свідчать про те, що це поле не буде корисне нам для подальшого прогнозування даних. 

Зробимо аналіз деяких показників.

* `````corr(Plan to Wath, Score)````` = `````0.48````` - Це найвищий показник для поля `````Score`````. Це свідчить про те, що більшість людей додають певний проект у свій список на перегляд, базуючись на тій оцінці, яку проекту поствили інші користувачі. 
* `````corr(Completed, Members)`````= `````0.98````` - Досить цікавою є висока залежність загальної аудиторії проекту та аудиторії, що повністю його додивилася, особливо у порівнянні з іншими статусами перегляду, такими як `````Watching````` чи `````Dropped`````. Це може означати що процент людей, що додивляються проект до кінця, дуже схожий для більшості проектів.
* `````corr(Members, Score-8)````` ... `````corr(Members, Score-1)````` - Чим більше у проекту більш високих оцінок, тим більше аудиторія цього проекту. Що цілком відповідає вже раніше отриманим даним.
* `````corr(Members, Score-10)`````, `````corr(Members, Score-9)````` - Тим не менш якщо розглянути залежності загальної аудиторії від найвищих оцінок, 9 та 10, наша зростаюча послідовність збивається.
* `````corr(Completed, Score-10)````` ... `````corr(Completed, Score-1)````` - Ці залежності повторюють залежності поля `````Members````` від оцінок користувачів. Це прояв біль прямої залежності полів `````Completed````` та `````Members`````, що вже була розглянута вище.
* `````corr(Favourites, Score-10)````` = `````0.94`````- Досить очікуваний показник, адже є логічним той факт, що проект, який людина додала до категоріх улюблених, вона буде високо оцінювати.
* `````corr(MAL_ID, Year)````` = `````0.59`````- це значення не має такого значення для нашого дослідження, проте добре показує що база даних є "живою" і поповнюється новими проектами по мірі їх виходу.



### 1.2.5. Конструювання ознак

Видалимо наступні поля:

Через непотрібність трьох полів для назви:
* `````English name`````
* `````Japanese name`````

Через те, що інформацію в них містять інші поля:
* `````Aired`````
* `````Score-1-10`````
* `````Ranked`````
* `````Popularity `````

Через завелику кількість невідомих значень:
* `````Premiered`````
* `````Producers`````
* `````Licensors`````

Через неспівставну складність отримання з цього поля дані зручного формату у порівнянні з потенційною корисністю:
* `````Duration`````

Через замалу корисність для майбутнього прогнозування за корреляційною таблицею:
* `````Episodes`````


In [ ]:
df.drop(["English name", "Japanese name", "Episodes", "Aired", "Premiered", "Producers", "Licensors", "Duration", "Ranked", "Popularity", "Score-1", "Score-2", "Score-3", "Score-4", "Score-5", "Score-6", "Score-7", "Score-8", "Score-9", "Score-10"], axis=1, inplace=True)
df.info()

Виконаємо кодування категоріальних типів даних. Для початку закодуємо більш прості поля, що містять лише одну категорію на запис використовуючи `````onehot encoding`````.

In [ ]:
df = pd.get_dummies(df, columns=["Type"])
df = pd.get_dummies(df, columns=["Source"])
df = pd.get_dummies(df, columns=["Rating"])
df = pd.get_dummies(df, columns=["Season"]) 

df.info()

Перейдемо до кодування полів, що можуть містити декілька категорій на запис. Для початку, переведемо значення цих полів до формату списку.

In [ ]:
df["Genders"] = df.Genders.apply(lambda x: x.split(','))
df["Studios"] = df.Studios.apply(lambda x: x.split(','))

df

In [ ]:
print(len(genres))
print(len(studios))

Тепер можна приступати до кодування даних.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

def list_code(df, name):
    df = df.join(
                pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(df.pop(name)),
                    index=df.index,
                    columns=mlb.classes_))
    return df

df = list_code(df, "Genders")
df = list_code(df, "Studios")


In [ ]:
df.info()

In [ ]:
df

### 1.2.7. Розбиття на тренувальний та тестовий набори даних

Наостанок, підготуємо тренувальний та тестовий набори даних для подальшої роботи.

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.33, random_state=42)

df.to_csv('df.csv', index=False)
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

### 1.2.7. Бізнес-профіль Еріксона-Пенкера.

In [ ]:
from IPython.display import Image
Image(filename='/kaggle/input/photos/ericsson-penker.png')

## 1.3. Верифікація і валідація
### 1.3.1 Верифікація
Головною ціллю даної лабораторної роботи була підготовка даних для вирішення певної конкретної задачі. В рамках виконання цієї лабораторної роботи потрібно було визначити предметну область, поставити контретну задачу, яку необхідно буде вирішити, дослідити дані, що були отримані із зовнішніх ресурсів та обрані у відповідністю з встановленою предметною областю, та підготувати ці дані для вирішення поставленої задачі.

Предметною областю була вибрана індустрія японської мультиплікації. Була поставлена задача, що полягає у побудові системи прогнозування рейтингу(оцінки) проекту на основі наявних даних, використовуючи досліджені залежності. Дані було успішно завантажено з ресурсу `````Kaggle````` та збережено, структуру таблиці було описано.

Була проведена підготовка даних до аналізу, а саме:

* Приведення числових даних до стандартних форматів
* Усунення дублікатів
* Усунення NaN та Null значень
* Виявлення та усунення "Unknown" значень

Було проведено розвідувальний аналіз даних, що включав у себе як застосування описової статистики, так і проведення  кореляційного  та  причинно-наслідкового  аналізів, побудовано відповідні графіки та вказано роз'яснення до них. Були сконструйовані нові ознаки  `````Eng_name_availability`````, `````Year````` та`````Season`````. Категоріальні ознаки було успішно закодовано, поля, що не представляли цінності для майбутнього прогнозування, було видалено.

Дані було розбито на тренувальний та тестовий набори. Також було задокументовано класи бізнес-профілю Еріксона-Пенкера.

Умови лабораторної роботи було виконано, тож верифікація пройшла успішно.

### 1.3.2 Валідація
Проаналізувавши датасет, можемо прийти до висновку, що дані є цілком коректними. Було отримано та досліджено результати, які можна назвати достовірними та такими, що відповідають реальності. Так, наприклад, найбільш поширеними жанрами виявилися найбільш загальні, такі як комедія, фантастика чи пригоди, а більш нішеві, як от історичні чи на тематику космосу, навпаки мали відносно малу поширеність. Іншим прикладом може бути дуже рівномірний розподіл виходу проектів у кожний сезон року. Отримані результати не суперечать очікуваним показникам в кожному з досліджень.

Головною ціллю дослідження є розробка моделі, що буде передбачувати рейтинг проектів за іншими характеристиками. Було виявлено, досить високу залежність поля `````Score````` від інших полів. Високій ймовірній точності передбвчень сприяє також велика кількість даних, що можуть бути використані для прогнозування. Тож приходимо до висновку, що передбачення, засноване на побудованій моделі, повинно бути досить точним та достовірним.

## 1.4 Висновки
За результатами верифікації можемо прийти до висновку, що робота відповідає заданим вимогам. Також в процесі виконання лабораторної роботи було опановано основні інструменти роботи з даними:

* pandas
* numpy
* matplotlib
* seaborn
* sklearn
* collections

Було пройдено основні етапи конструювання систем Data Science:

* Збирання сирих даних
* Очистка даних та первинна підготовка
* Розвідувальний аналіз даних
* Розбиття даних на тренувальні та тестові набори
* Верифікація та валідація

Отже, у ході лабораторної роботи дані були успішно підготовлені до наступного етапу розробки системи - застосування Machine Learning алгоритмів для прогнозування рейтингу(оцінки) проектів.